In [ ]:
import numpy as np
import underworld as uw
import glucifer

In [ ]:
# create mesh and deform (temperature-like field)
(Lx, Ly) = (10.,10.)
mesh = uw.mesh.FeMesh_Cartesian(elementType=('Q1'),elementRes=(8,8), maxCoord=(Lx,Ly))
deform = uw.mesh.MeshVariable(mesh,nodeDofCount=1)
# create 2nd mesh for visualisation
mesh2 = uw.mesh.FeMesh_Cartesian(elementType=('Q1'),elementRes=(8,8), maxCoord=(Lx,Ly))

In [ ]:
fig1 = glucifer.Figure(rulers=True, rulerticks=5)
fig1.append(glucifer.objects.Mesh(mesh,nodeNumbers=False, colour='blue'))
fig1.show()

In [ ]:
# define top as 1, bottom as 0
top = mesh.specialSets["MaxJ_VertexSet"]
bottom = mesh.specialSets["MinJ_VertexSet"]
deformBCs = uw.conditions.DirichletCondition(deform, indexSetsPerDof=(top+bottom))

deform.data[top.data] = 1.0
deform.data[bottom.data] = 0.0

heatEq = uw.systems.SteadyStateHeat(deform, fn_diffusivity=1.0, conditions=deformBCs)
solver = uw.systems.Solver(heatEq)
solver.solve()

In [ ]:
# copy initial deform data
initial = deform.data.copy()

In [ ]:
# deform the top surface with an arbitrary displacement, less than half cell width is best
with mesh.deform_mesh():
    for ind in top:
        mesh.data[ind][1] += .250*np.sin(2.*np.pi*mesh.data[ind][0]/Lx)

# duplicate new mesh
with mesh2.deform_mesh():
    mesh2.data[:] = mesh.data[:]

In [ ]:
fig1.append(glucifer.objects.Mesh(mesh2,nodeNumbers=False, colour='red'))
fig1.show()

In [ ]:
# solve steadyStateEq again
solver.solve()

In [ ]:
# get new deform values
new = deform.evaluate(mesh.data)

In [ ]:
# get the non-zero temperature nodes
inds = np.where(initial>0)[0]
with mesh.deform_mesh():
    # scale node ycoords with ratio of initial/new
    mesh.data[inds,1] = mesh.data[inds,1]*(initial[inds]/new[inds]).T

In [ ]:
fig1.show()